In [1]:
import pandas as pd
import numpy as np

In [2]:
sf_permits = pd.read_csv("../dataset/building-permit-applications-data/Building_Permits.csv")

/home/neil/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (22,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df = sf_permits[['Current Status','Filed Date','Issued Date','Zipcode','Supervisor District']]

In [4]:
zips = pd.get_dummies( df['Zipcode'] , prefix = 'z' )

In [5]:
df=pd.concat([df, zips], axis=1)
sds = pd.get_dummies( df['Supervisor District'] , prefix = 'sd' )
df=pd.concat([df, sds], axis=1)
df=df.drop(columns=['Zipcode','Supervisor District'])

In [6]:
df['from_date']=pd.to_datetime(df['Filed Date'])
df['to_date']=pd.to_datetime(df['Issued Date'])
#df['process_time']=df['to_date']-df['from_date']
#df['total_time']=df['process_time'].astype('timedelta64[D]')
#df=df.drop(columns=['process_time'])
df=df.drop(columns=['Filed Date','Issued Date'])

In [7]:
import datetime as dt
df['from_date']=df['from_date'].map(dt.datetime.toordinal)
df['to_date']=df['to_date'].map(dt.datetime.toordinal)

In [8]:
#df

In [9]:
df['process_time']=df['to_date']-df['from_date']
df['total_time']=df['process_time'].astype('timedelta64[D]')

In [10]:
df=df.drop(columns=['total_time'])

In [11]:
df

Current Status  z_94102.0  z_94103.0  z_94104.0  z_94105.0  z_94107.0  \
0             expired          1          0          0          0          0   
1              issued          1          0          0          0          0   
2           withdrawn          0          0          0          0          0   
3            complete          0          0          0          0          0   
4              issued          1          0          0          0          0   
5              issued          0          0          0          0          1   
6            complete          0          0          0          0          0   
7              issued          0          0          0          0          0   
8              issued          0          0          0          0          0   
9              issued          0          0          0          0          0   
10             issued          0          0          0          0          0   
11             issued          1          0          0          0          0   
12             issued          0          0          0          0          0   
13              filed          0          0          0          0          0   
14             issued          0          0          0          0          0   
15             issued          0          0          0          0          0   
16             issued          0          0          1          0          0   
17             issued          0          0          0          0          0   
18              filed          0          0          0          0          0   
19             issued          0          0          0          0          0   
20             issued          0          0          0          0          0   
21              filed          0          0          0          0          0   
22              filed          0          0          0          0          0   
23             issued          0          0          0          0          0   
24             issued          0          0          0          0          0   
25             issued          0          0          0          0          0   
26             issued          0          0          0          0          0   
27             issued          0          0          0          0          0   
28             issued          0          0          0          0          0   
29             issued          1          0          0          0          0   
...               ...        ...        ...        ...        ...        ...   
198870         issued          0          0          0          0          0   
198871         issued          0          0          0          0          0   
198872          filed          0          0          0          0          0   
198873         issued          0          0          0          0          0   
198874         issued          0          0          0          0          0   
198875          filed          1          0          0          0          0   
198876         issued          0          0          0          0          0   
198877         issued          0          0          0          0          0   
198878          filed          0          0          0          0          0   
198879         issued          0          0          0          0          0   
198880         issued          0          0          0          0          0   
198881         issued          0          0          0          0          0   
198882         issued          0          0          0          0          0   
198883         issued          1          0          0          0          0   
198884          filed          0          1          0          0          0   
198885          filed          0          0          0          0          0   
198886       approved          0          0          0          0          0   
198887       complete          0          0          0          0          0   
198888 

In [12]:
df_filed = df.loc[df['Current Status'].isin(['filed'])]
df_completed = df.loc[df['Current Status'].isin(['complete','issued'])]
df_completed=df_completed.drop(columns=['Current Status'])
df_filed=df_filed.drop(columns=['Current Status','to_date'])

In [13]:
df_completed=df_completed[df_completed['to_date']!=1]

In [14]:
df_completed=df_completed.drop(columns=['to_date'])

In [15]:
df_c2=df_completed.dropna()

In [16]:
df_c2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180618 entries, 1 to 198899
Data columns (total 40 columns):
z_94102.0       180618 non-null uint8
z_94103.0       180618 non-null uint8
z_94104.0       180618 non-null uint8
z_94105.0       180618 non-null uint8
z_94107.0       180618 non-null uint8
z_94108.0       180618 non-null uint8
z_94109.0       180618 non-null uint8
z_94110.0       180618 non-null uint8
z_94111.0       180618 non-null uint8
z_94112.0       180618 non-null uint8
z_94114.0       180618 non-null uint8
z_94115.0       180618 non-null uint8
z_94116.0       180618 non-null uint8
z_94117.0       180618 non-null uint8
z_94118.0       180618 non-null uint8
z_94121.0       180618 non-null uint8
z_94122.0       180618 non-null uint8
z_94123.0       180618 non-null uint8
z_94124.0       180618 non-null uint8
z_94127.0       180618 non-null uint8
z_94129.0       180618 non-null uint8
z_94130.0       180618 non-null uint8
z_94131.0       180618 non-null uint8
z_94132.0      

In [17]:
# ML Algorithm
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

#reg=linear_model.LinearRegression()
rg = DecisionTreeRegressor()

In [18]:
df_x=df_c2.drop(columns=['process_time'])
df_y=df_c2['process_time']

In [19]:
df_x

z_94102.0  z_94103.0  z_94104.0  z_94105.0  z_94107.0  z_94108.0  \
1               1          0          0          0          0          0   
3               0          0          0          0          0          0   
4               1          0          0          0          0          0   
5               0          0          0          0          1          0   
6               0          0          0          0          0          0   
7               0          0          0          0          0          0   
8               0          0          0          0          0          0   
9               0          0          0          0          0          0   
10              0          0          0          0          0          0   
11              1          0          0          0          0          0   
12              0          0          0          0          0          0   
14              0          0          0          0          0          0   
15              0          0          0          0          0          1   
16              0          0          1          0          0          0   
17              0          0          0          0          0          0   
19              0          0          0          0          0          0   
20              0          0          0          0          0          0   
23              0          0          0          0          0          0   
24              0          0          0          0          0          0   
25              0          0          0          0          0          0   
26              0          0          0          0          0          0   
27              0          0          0          0          0          0   
28              0          0          0          0          0          0   
29              1          0          0          0          0          0   
30              0          0          0          0          0          0   
31              0          0          0          0          0          0   
32              0          0          0          0          0          0   
34              0          1          0          0          0          0   
35              0          0          0          0          0          0   
36              0          0          0          0          0          0   
...           ...        ...        ...        ...        ...        ...   
198858          0          0          0          0          0          0   
198861          0          0          0          0          0          1   
198863          0          0          0          0          0          0   
198865          0          0          0          0          0          0   
198867          0          0          0          0          1          0   
198868          0          0          0          0          0          0   
198869          0          0          0          0          0          0   
198870          0          0          0          0          0          0   
198871          0          0          0          0          0          0   
198873          0          0          0          0          0          0   
198874          0          0          0          0          0          0   
198876          0          0          0          0          0          0   
198877          0          0          0          0          0          0   
198879          0          0          0          0          0          0   
198880          0          0          0          0          0          0   
198881          0          0          0          0          0          0   
198882          0          0          0          0          0          0   
198883          1          0          0          0          0          0   
198887          0          0          0          0          0          0   
198888          0          0          0          0          0          0   
198889          0          0          0          0      

In [20]:
x_train=df_x[:130618]
x_test=df_x[130618:]
y_train=df_y[:130618]
y_test=df_y[130618:]

In [21]:
model = rg.fit(x_train,y_train)

In [23]:
#reg.coef_

In [24]:
a=rg.predict(x_test)

In [25]:
model.score(x_test,y_test)

-0.3874236507018472

In [26]:
from sklearn.metrics import mean_squared_error

In [27]:
mean_squared_error(y_test,a)

13355.370124038642